In [20]:
import pandas as pd



In [21]:
df_path = "/data/gc/rocksdb-index/openPMD-viewer/results/500g_iteration_10000/benchmark_result_selected_15_test.csv"

df = pd.read_csv(df_path, header=[0])

In [22]:
# print all df columns
print(df.columns)

Index(['query_seq', 'target_percentage', 'select_set', 'test_type',
       'test_name', 'index_type', 'storage', 'secondary', 'direct_block_read',
       'read_groups', 'skip_offset', 'query_index_time_elapsed',
       'remove_duplication_time_elapsed', 'sort_block_metadata_time_elapsed',
       'find_optimal_read_solution_time_elapsed',
       'generate_select_array_time_elapsed', 'get_target_data_time_elapsed',
       'get_support_data_time_elapsed', 'data_calculation_time_elapsed',
       'data_apply_select_time_elapsed',
       'apply_particle_level_select_array_time_elapsed', 'total_time_elapsed',
       'query_result_size', 'chunk_range_size', 'current_percentage',
       'iteration', 'species', 'expand_set', 'envelope', 'data_size'],
      dtype='object')


In [23]:
# deduplication and keep the last by the column [query_seq target_percentage select_set  test_type test_name index_type storage secondary direct_block_read read_groups envelope]
df = df.drop_duplicates(subset=['test_type', 'envelope'], keep='last')

In [24]:

# remove the row that 'data_size' is NaN
df = df.dropna(subset=['data_size'])



In [25]:
# sort by query_seq target_percentage test_type
df = df.sort_values(by=['target_percentage', 'envelope', 'test_type'])



In [26]:
# reset index
df = df.reset_index(drop=True)

In [27]:
# keep the rows, where after grouping the df by envelope, if the group equals to 5
df = df.groupby('envelope').filter(lambda x: len(x) == 5)

In [28]:
# print the count after grouping by target_percentage and select_set, and then count the size of each group / 5
print(df.groupby(['target_percentage', 'select_set']).size() / 5)

target_percentage  select_set                  
0.01%              ('ux','uy')                     100
                   ('ux','uy','uz')                 55
                   ('ux','uy','uz','x')            100
                   ('ux','uy','uz','x','y')        100
                   ('ux','uy','uz','x','y','z')    160
                   ('ux','uy','x','y')             100
                   ('ux',)                          50
                   ('x','y')                       100
                   ('x','y','z')                    70
                   ('x',)                           60
dtype: int64


In [19]:
# save to csv
df.to_csv(df_path.replace('before', 'after').replace('test', 'fin'), index=False)